Code to implement and test changes to icesat2 download structure so that it is compliant with the new Harmony backend

In [1]:
import os
import datetime

import geopandas as gpd

from HydroEO.utils import geometry
from harmony import WKT, Client, Collection, Request

In [2]:
# load example data
gdf = gpd.read_file(
    r"C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\notebooks\example_data\example_res.shp"
)
gdf.head()

,project,geometry
0,Nuozhadu,"POLYGON ((100.31926 22.97188, 100.31983 22.971..."


In [3]:
# grab coordinates of geometry
coords = [(x, y) for x, y in gdf.loc[0, "geometry"].envelope.exterior.coords]

# set start and end dates
startdate = datetime.date(*(2019, 1, 1))
enddate = datetime.date(*(2025, 1, 1))

In [4]:
token = os.getenv("EDL_TOKEN")
if token is None or token == "":
    print("Use .netrc for client authentication")
    harmony_client = Client()
else:
    print("Use EDL token from the environment for client authentication")
    harmony_client = Client(token=token)

Use .netrc for client authentication


In [5]:
# query and download data
# print(f"Searching for Icesat2 ATL13 for aoi from {startdate} to {enddate}")
# _ = icesat2.query(
#     aoi=coords,
#     startdate=startdate,
#     enddate=enddate,
#     earthdata_credentials=credentials,
#     download_directory=download_dir,
#     product="ATL13",
# )

# request the
request = Request(
    collection=Collection(id="ATL13"),
    # spatial=BBox(-155.75, 19.26, -155.3, 19.94), # bounding box example that can be used as an alternative to shapefile input
    # spatial=WKT('POLYGON((-155.75 19.26, -155.3 19.26, -155.3 19.94, -155.75 19.94, -155.75 19.26))'), # WKT example
    spatial=WKT(geometry.poly_coord_list_to_wkt(coords)),
    temporal={
        "start": startdate,
        "stop": enddate,
    },
    # variables=["science/grids/data/unwrappedPhase"],
    # format="HDF5",
    # max_results=2,
    # If desired, deliver results to a custom destination bucket. Note the bucket must reside in AWS us-west-2 region.
    # destination_url='s3://my-bucket'
)

In [6]:
request.is_valid()

True

In [7]:
job_id = harmony_client.submit(request)
job_id

'26d6d464-97b3-4aac-abb2-1383f3ae87d8'

In [8]:
harmony_client.wait_for_processing(job_id, show_progress=True)

 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   9% ] |####                                               | [-]
 [ Processing:   9% ] |####                                               | [\]
 [ Processing:   9% ] |####                                               | [|]
 [ Processing:   9% ] |####             

In [9]:
data = harmony_client.result_json(job_id)
data

{'username': 'connorchewning',
 'status': 'complete_with_errors',
 'message': 'The job has completed with errors. See the errors field for more details',
 'progress': 100,
 'createdAt': '2025-02-13T09:19:14.012Z',
 'updatedAt': '2025-02-13T09:26:37.300Z',
 'dataExpiration': '2025-03-15T09:19:14.012Z',
 'links': [{'title': 'STAC catalog',
   'href': 'https://harmony.earthdata.nasa.gov/stac/26d6d464-97b3-4aac-abb2-1383f3ae87d8/',
   'rel': 'stac-catalog-json',
   'type': 'application/json'},
  {'href': 'https://harmony.earthdata.nasa.gov/service-results/harmony-prod-staging/public/26d6d464-97b3-4aac-abb2-1383f3ae87d8/89626243/ATL13_20190813181336_07130401_006_01_subsetted.h5',
   'title': 'ATL13_20190813181336_07130401_006_01_subsetted.h5',
   'type': 'application/x-hdf5',
   'rel': 'data',
   'bbox': [-180, -90, 180, 90],
   'temporal': {'start': '2019-08-13T18:13:36.000Z',
    'end': '2019-08-14T00:30:46.000Z'}},
  {'href': 'https://harmony.earthdata.nasa.gov/service-results/harmony-pr

In [10]:
urls = harmony_client.result_urls(job_id, show_progress=True)
urls

<generator object Client.result_urls at 0x000001D0AB8B7740>

In [11]:
results = harmony_client.download_all(
    job_id,
    directory=r"C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp",
    overwrite=True,
)
file_names = [f.result() for f in results]
file_names

C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626243_ATL13_20190813181336_07130401_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626234_ATL13_20190514223354_07130301_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626253_ATL13_20191112135332_07130501_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626251_ATL13_20191022022631_03850501_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626260_ATL13_20200120220613_03850601_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626280_ATL13_20200720132550_03850801_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626224_ATL13_20190213025414_07130201_006_01_subsetted.h5
C:\Users\cwch\Projects\WR-Collaboration\rk-altimetry\data\icesat2\temp\89626222_ATL13_2019

['C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626243_ATL13_20190813181336_07130401_006_01_subsetted.h5',
 'C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626234_ATL13_20190514223354_07130301_006_01_subsetted.h5',
 'C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626253_ATL13_20191112135332_07130501_006_01_subsetted.h5',
 'C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626251_ATL13_20191022022631_03850501_006_01_subsetted.h5',
 'C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626260_ATL13_20200120220613_03850601_006_01_subsetted.h5',
 'C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626280_ATL13_20200720132550_03850801_006_01_subsetted.h5',
 'C:\\Users\\cwch\\Projects\\WR-Collaboration\\rk-altimetry\\data\\icesat2\\temp\\89626224_ATL13_20190213025414_07130201_006_01_subsetted.h5',

In [15]:
from HydroEO.satellites import icesat2

In [16]:
keys = ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
data = icesat2.ATL13(file_names[0], keys[0])
df = data.read()

In [17]:
df

,height,lat,lon,date,wb_type,wb_size,wb_id,dem,sat_frac_track,beam,orbit,rgt,cycle_number,file_name
0,773.765991,22.841353,100.424546,2019-08-13 18:19:51.592502,5,0,2006439,720.598267,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
1,773.734863,22.841545,100.424532,2019-08-13 18:19:51.595502,5,0,2006439,730.037415,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
2,773.790649,22.848726,100.423977,2019-08-13 18:19:51.707302,5,0,2006439,610.340332,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
3,773.775024,22.849302,100.423929,2019-08-13 18:19:51.716402,5,0,2006439,619.576965,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
4,773.765076,22.850232,100.423852,2019-08-13 18:19:51.731002,5,0,2006439,637.384888,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
5,773.734497,22.850946,100.423789,2019-08-13 18:19:51.742102,5,0,2006439,653.239990,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
6,773.752808,22.851294,100.423757,2019-08-13 18:19:51.747502,5,0,2006439,659.270142,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
7,773.818359,22.851641,100.423726,2019-08-13 18:19:51.752902,5,0,2006439,665.243286,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
8,773.792419,22.851898,100.423703,2019-08-13 18:19:51.756902,5,0,2006439,670.954956,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
9,773.805420,22.852298,100.423669,2019-08-13 18:19:51.763202,5,0,2006439,677.702515,0.0,gt1l,5075,713,4,89626243_ATL13_20190813181336_07130401_006_01_...
